1º Parte: Receber o arquivo

In [3]:
#@title Receber o video no colab (Upload)
from google.colab import files
uploaded = files.upload() # Até ~100MB

Saving BATALHA DOS MEMES.mp4 to BATALHA DOS MEMES.mp4


In [ ]:
#@title Receber o video da database
# https://www.kaggle.com/datasets/kunwardeepak/youtube-trending-august-2020

2º Parte: Processar o arquivo (audio e frames)

In [4]:
#@title Verificar a duração do video para restrições de frames e audio
!pip install ffmpeg
import ffmpeg
import subprocess

# Retorna a duração em segundos
def verificarDuracaoVideo(videoEntrada):
  try:
    commando = [
      'ffprobe',
      '-i', videoEntrada,
      '-show_entries', 'format=duration',
      '-v', 'quiet',
      '-of', 'csv=p=0'
    ]
    # Executar o comando
    resultado = subprocess.run(commando, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    stderr = resultado.stderr.decode('utf-8')
    if stderr:
      print(f"Erro do ffprobe: {stderr}")
    # Ler a duração
    duracao = float(resultado.stdout.decode('utf-8').strip())
    return duracao
  except Exception as e:
    print(f"Erro ao converter a duração: {e}")
    return None


videoEntrada = None
# Encontrar o video
import os
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  # Verificar a duração do video
  duracaoVideo = verificarDuracaoVideo(videoEntrada)
  print(videoEntrada,duracaoVideo)
else:
  print('Nenhum video encontrado!')

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=6b7328624718e25ad52a4cfef6857f5099b97efb35337dd86328d99facf38a0d
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg
BATALHA DOS MEMES.mp4 134.49


In [5]:
#@title Extrair o audio do video
#!pip install ffmpeg-python # redundante

import ffmpeg
import subprocess

def extrair_audio(videoEntrada, localAudioSaida):
  try:
    subprocess.run([
      'ffmpeg', '-i', videoEntrada,
      '-vn', '-acodec', 'libmp3lame', '-ab', '192k', localAudioSaida
    ], check=False)
    print("Audio extraido!")
  except Exception as e:
    print(f"Erro: {e}")

videoEntrada = None
# Encontrar o video
import os
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  localAudioSaida = videoEntrada.replace('.mp4', '.mp3')
  extrair_audio(videoEntrada, localAudioSaida)
else:
  print('Nenhum video encontrado!')

Audio extraido!


In [ ]:
#@title Se necessário, repartir o audio em partes menores
#!pip install ffmpeg-python # redundante
import ffmpeg
import os

def dividirAudio(audioEntrada, localAudioSaida, tamanhoMaximo = None):
  # Se não existir -> criar a pasta
  if not os.path.exists(localAudioSaida):
    os.makedirs(localAudioSaida)
  # Tentar dividir o audio
  try:
    ffmpeg.input(audioEntrada).output(f"{localAudioSaida}/chunk%03d.mp3", f='segment', segment_time=tamanhoMaximo).run()
    print("Audio dividido!")
  except ffmpeg.Error as e:
    print(f"Erro: {e.stderr.decode()}")

# Encontrar o audio
audioEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp3"):
    audioEntrada = arquivo
    break

#if audioEntrada:
#  dividirAudio(audioEntrada,'/content/Audio/', 2*60)
#else:
#  print('Nenhum audio encontrado!')

In [6]:
#@title Extrair os frames do video
!pip install opencv-python
import cv2
import time
import os

def extrairFrames(videoEntrada, localFramesSaida):
  try:
    # Se não existir -> criar a pasta
    if not os.path.exists(localFramesSaida):
      os.makedirs(localFramesSaida)

    start_time = time.time() # Para o cronometro

    cap = cv2.VideoCapture(videoEntrada)
    frame_count = 0
    frame_counter = 0
    # Determinar intervalo da captura
    fps = 1
    duracaoMaximaVideo = 5*60
    intervaloExtracao = 1.0/fps
    if duracaoVideo > duracaoMaximaVideo:
      intervaloExtracao = duracaoVideo/(fps*duracaoMaximaVideo)
    # Video em si
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # Get timestamp in seconds

      # Salvar o frame a cada {intervaloExtracao}
      if current_time >= frame_count * intervaloExtracao:
          frame_filename = os.path.join(localFramesSaida, f"frame_{frame_counter:04}_time_{current_time:.2f}.png")
          cv2.imwrite(frame_filename, frame)
          frame_counter += 1
          frame_count += 1

      # Break para quando o video ter acabado
      if current_time > cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0:
          break

    cap.release()
    end_time = time.time() # Parar o cronometro
    print('Frames extraidos')
    print(f"Tempo de execução: {end_time - start_time: .1f} segundos")
    print(f"{(frame_counter+1)/(end_time-start_time): .2f} frames por segundo")
  except Exception as e:
    print(f"Erro: {e}")

# Encontrar o video
videoEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  extrairFrames(videoEntrada, '/content/Frames/Original/')
else:
  print('Nenhum video encontrado!')

Frames extraidos
Tempo de execução:  42.1 segundos
 3.23 frames por segundo


In [7]:
#@title Processar os frames
!pip install Pillow
from PIL import Image

def processarFrames(localFrames):
  try:
    # Se não existir -> criar a pasta
    if not os.path.exists('/content/Frames/Processados/'):
        os.makedirs('/content/Frames/Processados/')
    # Tentar processar os frames
    for arquivo in os.listdir(localFrames):
      if arquivo.endswith('.png'):
        # Imagem recebida
        with Image.open(os.path.join(localFrames,arquivo)) as img:
          # Verificando tamanho da imagem
          width, height = img.size
          if width > 1280 or height > 720: # Reduzir tamanho
            # Seguir proporção para evitar distorção
            proporcao = width/height
            if proporcao > 1: # Paisagem
              img = img.resize((1280, int(1280/proporcao)), Image.Resampling.LANCZOS)
            else: # Retrato
              img = img.resize((int(720*proporcao), 720), Image.Resampling.LANCZOS)

            # Salvar imagem na nova pasta
            img.save(os.path.join('/content/Frames/Processados/', arquivo))
    print('Frames processados')
  except Exception as e:
    print(f"Erro: {e}")

# Tentar processar os frames
import os
if os.path.exists('/content/Frames/Original/'):
  processarFrames('/content/Frames/Original/')
else:
  print('Nenhum frame encontrado!')

Frames processados


In [8]:
#temp
!clear
import os
print(os.listdir())
if os.path.exists('/content/Frames/Original/'):
  print(sorted(os.listdir('/content/Frames/Original/')))
  print(len(os.listdir('/content/Frames/Original/')),end='\n------\n')
if os.path.exists('/content/Frames/Processados/'):
  print(sorted(os.listdir('/content/Frames/Processados/')))
  print(len(os.listdir('/content/Frames/Processados/')),end='\n------\n')

['.config', 'BATALHA DOS MEMES.mp4', 'BATALHA DOS MEMES.mp3', 'Frames', 'sample_data']
['frame_0000_time_0.00.png', 'frame_0001_time_1.00.png', 'frame_0002_time_2.00.png', 'frame_0003_time_3.00.png', 'frame_0004_time_4.00.png', 'frame_0005_time_5.00.png', 'frame_0006_time_6.01.png', 'frame_0007_time_7.01.png', 'frame_0008_time_8.01.png', 'frame_0009_time_9.01.png', 'frame_0010_time_10.01.png', 'frame_0011_time_11.01.png', 'frame_0012_time_12.01.png', 'frame_0013_time_13.01.png', 'frame_0014_time_14.01.png', 'frame_0015_time_15.01.png', 'frame_0016_time_16.02.png', 'frame_0017_time_17.02.png', 'frame_0018_time_18.02.png', 'frame_0019_time_19.02.png', 'frame_0020_time_20.02.png', 'frame_0021_time_21.02.png', 'frame_0022_time_22.02.png', 'frame_0023_time_23.02.png', 'frame_0024_time_24.02.png', 'frame_0025_time_25.02.png', 'frame_0026_time_26.03.png', 'frame_0027_time_27.03.png', 'frame_0028_time_28.03.png', 'frame_0029_time_29.03.png', 'frame_0030_time_30.03.png', 'frame_0031_time_31.03.

3º Parte: Criar descrição e transcrição dos frames e áudio, respectivamente

In [9]:
### Instalar as bibliotecas maiores
# 1~3 minutos
!pip install openai-whisper
!pip install torch transformers pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s

In [10]:
#@title Transcrição do audio (Whisper)
!pip install openai-whisper
import whisper

def transcreverAudio(audioEntrada):
  try:
    # Tentar transcrever o audio
    model = whisper.load_model("base")
    result = model.transcribe(audioEntrada, language='pt')
    print("Audio Extraido!")
    return result["text"]
  except Exception as e:
    print(f"Erro: {e}")
    return None

audioEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp3"):
    audioEntrada = arquivo
    break

if audioEntrada:
  transcricaoAudio = None # Salvar para caso de erro
  transcricaoAudio = transcreverAudio(audioEntrada)
else:
  print('Nenhum audio encontrado!')

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 54.4MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Audio Extraido!


In [11]:
#@title Descrição dos frames
## 135 frames em 14 minutos
## Média de 5~7 segundos por frame
!pip install torch transformers pillow

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import os

pastaFrames = "/content/Frames/Processados/" # Tentar usar os frames já processados

try:
    # Tenta configurar o modelo
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    # Descrição de 1 frame
    def descrever_frame(caminho_imagem):
        image = Image.open(caminho_imagem).convert("RGB")

        # Processa a imagem com o processador BLIP, retornando os tensores necessários para o modelo
        inputs = processor(image, return_tensors="pt")

        # Gera a descrição da imagem a partir do modelo BLIP (max_length=100 significa que a descrição terá no máximo 100 tokens)
        output = model.generate(**inputs, max_length=100)

        # Decodifica a saída do modelo e converte para uma string legível
        descricao = processor.decode(output[0], skip_special_tokens=True)
        return descricao

    # Lista com a descrição dos frames
    def descreverFramesVideo(localFrames = '/content/Frames/Processados/'):
      import time
      listaDescricaoFrames = [] # List para armazenar as descrições
      # Carregar vários frames
      start_time = time.time()
      counter = 0
      for frame in sorted(os.listdir(localFrames)):
        if frame.endswith((".jpg", ".png")):
          # Mandar mais feedback para o usuário
          counter += 1
          if counter % 10 == 0:
            print(f"{counter} frames processados em {int((time.time()-start_time)/60)}min {(time.time()-start_time)%60: .1f}s")

          caminho = os.path.join(localFrames, frame)
          descricao = descrever_frame(caminho)
          # Dict para também armazenar o frame (por consequencia o timestamp)
          infoFrame = {
              'frame': frame,
              'descricao': descricao,
          }
          listaDescricaoFrames.append(infoFrame)
      end_time = time.time()
      print('Descrição de frames criada!')
      print(f"{(end_time-start_time)/len(listaDescricaoFrames): .2f} segundos por frame")
      return listaDescricaoFrames

except Exception as e:
    print(f"Erro: {e}")

# Usar original se não foi necessário processar
if not os.path.exists(pastaFrames): # Frames processados não existem -> usar os originais
  pastaFrames = '/content/Frames/Original/'

if os.path.exists(pastaFrames):
  listDescricoesFrames = None # Salvar para caso de erro
  listDescricoesFrames = descreverFramesVideo(pastaFrames)
else:
  print('Nenhum video encontrado!')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

10 frames processados em 0min  54.2s
20 frames processados em 1min  43.9s
30 frames processados em 2min  34.8s
40 frames processados em 3min  28.0s
50 frames processados em 4min  20.6s
60 frames processados em 5min  10.1s
70 frames processados em 6min  1.1s
80 frames processados em 6min  54.4s
90 frames processados em 7min  46.5s
100 frames processados em 8min  32.4s
110 frames processados em 9min  20.8s
120 frames processados em 10min  22.2s
130 frames processados em 11min  19.4s
Descrição de frames criada!
 5.37 segundos por frame


4º Parte: Criação do sumário

4.1: Confirgurar o ollama

In [12]:
!pip install ollama

!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [21]:
!ps aux | grep ollama # Verificar processos rodando
!nohup ollama serve > ollama.log 2>&1 & ## Reset ollama


root        7990  0.0  0.0   7376  3536 ?        S    22:31   0:00 /bin/bash -c ps aux | grep ollama
root        7992  0.0  0.0   6484  2252 ?        S    22:31   0:00 grep ollama


In [15]:
%env OLLAMA_DEBUG=1
%env OLLAMA_NO_GPU=1

!ollama serve

env: OLLAMA_DEBUG=1
env: OLLAMA_NO_GPU=1
Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [16]:
!ollama pull mistral # Mais barato que LLaMA 3:8B

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏  18 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕▏  39 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  83 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 107 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 126 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 143 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   4% ▕▏ 166 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   4% ▕▏ 178 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   5% ▕▏ 206 MB/4.1 GB  206 MB/s     18spulling manife

In [22]:
!curl http://127.0.0.1:11434/api/tags


{"models":[{"name":"mistral:latest","model":"mistral:latest","modified_at":"2025-02-25T22:25:17.111563473Z","size":4113301824,"digest":"f974a74358d62a017b37c6f424fcdf2744ca02926c4f952513ddf474b2fa5091","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"7.2B","quantization_level":"Q4_0"}}]}

4.2: Criando os sumários usando ollama

In [18]:
#@title Tentativa de criação de contexto, criar uma descrição entre dois frames vizinhos (LLaMa)
# Baseado em https://github.com/GoloMarcos/LLM4BrazilianFakeNews/blob/main/Evaluation.ipynb
# 150~280 segundos por par
# Limite de 90 minutos de execução por causa do plano gratuito
import ollama
import time

# Fazer a analise entre a descrição de dois frames
def analisaVizinhosLlama(model, desc1, desc2):
  # Variaveis para refencia na prompt
  descricao1 = desc1['descricao']
  frame1_time = desc1['frame'].split('_time_')[1].split('.')[0]
  descricao2 = desc2['descricao']
  frame2_time = desc2['frame'].split('_time_')[1].split('.')[0]
  # prompt
  response = ollama.chat(model=model, messages=[{
    'role': 'user',
    'content': f'Considere as duas descrições em inglês a seguir e resuma-as em pt-br, contextualizando elas em conjunto. Explique como as duas se relacionam/diferenciam e o que elas podem significar em conjunto\nDescrição 1: {descricao1} instante {frame1_time}\nDescrição 2: {descricao2} instante {frame2_time}\nSeu resumo deve integrar as duas descrições e proporcionar uma visão clara e coerente do contexto geral.'
  },
  ])
  return response['message']['content']

# Criar a lista com as novas descrições
model = 'mistral'
def descricaoFramesVizinhosLlama(listDescricoesRecebida, indexInicio, indexFim):
  listDescricoesVizinhos = []
  start_time = time.time() # Cronometro
  for i in range(indexInicio, min(indexFim, (len(listDescricoesRecebida)-1))): # Percorrer até o indexFim ou o penultimo item da lista
    resposta = analisaVizinhosLlama(model, listDescricoesFrames[i], listDescricoesFrames[i+1]) # enviar par de descricoes
    # se recebido -> adicionar na lista
    if resposta:
      # Armazenar as informações dos frames e a nova descrição
      dictResposta = {
          'frame1': listDescricoesFrames[i]['frame'],
          'frame2': listDescricoesFrames[i+1]['frame'],
          'descricao': resposta,
      }
      listDescricoesVizinhos.append(dictResposta)
    # Um pouco mais de responsividade devido o tempo de execução
    print(f'{len(listDescricoesVizinhos)}/{len(listDescricoesRecebida)-1} em {int((time.time()-start_time)/3600)}h {int((time.time()-start_time)/60)%60}min {(time.time()-start_time)%60:.1f}s') # Cronometro
  return listDescricoesVizinhos

# Rodar as funções para permitir a chamada em outra célula
if listDescricoesFrames:
  tempoPrevisao = time.time()
  descricaoFramesVizinhosLlama(listDescricoesFrames, 0, 1)
  tempoPrevisao = time.time() - tempoPrevisao
else:
    print('Nenhuma descrição encontrada!')

1/134 em 0h 3min 39.1s


In [23]:
#@title Criar a lista de contexto com as restrições do colab gratuito
##### Primeiro rodar a celula acima
def criarSumarioVizinhosLLama(listaDescricoesOriginais):
  step = int((80*60)/tempoPrevisao) # reduzindo cada chunk para periodos de 80 minutos por segurança
  teto = 0
  resultadoCompleto = []
  while(teto < len(listaDescricoesOriginais)):
    teto += step
    resultadoParcial = descricaoFramesVizinhosLlama(listaDescricoesOriginais, (teto-step), teto)
    if resultadoParcial:
      resultadoCompleto += resultadoParcial
  return resultadoCompleto

if listDescricoesFrames:
  print(f'Previsao simplificada: {int((tempoPrevisao*(len(listDescricoesFrames)-1))/3600)}h {int((tempoPrevisao*(len(listDescricoesFrames)-1))/60)%60}min {(tempoPrevisao*(len(listDescricoesFrames)-1))%60:.1f}s')
  novoResultadoContexto = criarSumarioVizinhosLLama(listDescricoesFrames)
else:
    print('Nenhuma descrição encontrada!')

Previsao simplificada: 8h 9min 15.9s
1/134 em 0h 4min 23.7s
2/134 em 0h 7min 41.1s
3/134 em 0h 12min 26.6s
4/134 em 0h 15min 6.9s
5/134 em 0h 17min 45.2s
6/134 em 0h 21min 10.1s
7/134 em 0h 25min 56.1s
8/134 em 0h 29min 17.0s
9/134 em 0h 32min 34.9s
10/134 em 0h 35min 10.0s
11/134 em 0h 37min 48.6s
12/134 em 0h 41min 18.0s
13/134 em 0h 44min 8.2s
14/134 em 0h 46min 20.3s
15/134 em 0h 49min 12.8s
16/134 em 0h 52min 32.0s
17/134 em 0h 56min 38.5s
18/134 em 0h 59min 13.9s
19/134 em 1h 1min 43.9s
20/134 em 1h 4min 58.8s
21/134 em 1h 8min 14.7s
1/134 em 0h 4min 16.4s
2/134 em 0h 7min 13.0s
3/134 em 0h 10min 24.8s
4/134 em 0h 13min 0.3s
5/134 em 0h 16min 5.2s
6/134 em 0h 18min 51.7s
7/134 em 0h 23min 25.0s
8/134 em 0h 26min 38.0s
9/134 em 0h 30min 12.2s
10/134 em 0h 34min 27.3s
11/134 em 0h 38min 15.7s
12/134 em 0h 41min 40.4s
13/134 em 0h 45min 23.2s
14/134 em 0h 49min 48.2s
15/134 em 0h 52min 50.2s
16/134 em 0h 56min 1.0s
17/134 em 0h 58min 54.0s
18/134 em 1h 1min 37.9s
19/134 em 1h 5min 3

In [24]:
print(len(novoResultadoContexto))
for i in range(len(novoResultadoContexto)):
  print(novoResultadoContexto[i])

134
{'frame1': 'frame_0000_time_0.00.png', 'frame2': 'frame_0001_time_1.00.png', 'descricao': ' Em instante 0, um grupo de pessoas está se reunindo em volta de uma mesa numa sala. Em seguida, no instante 1, o grupo continua a sentar-se na mesma sala, porém não se especifica que estão em volta da mesa. Portanto, é possível inferir que em instante 0, o grupo foi convocado para a sala e começou a se reunir em volta de uma mesa, após isso, no instante 1, eles se mantiveram sentados na mesma sala mas não necessariamente em volta da mesa. Em conjunto, estas descrições mostram que um grupo está reunindo-se numa sala para algum propósito, possivelmente conversando ou realizando trabalho coletivo, e ao longo do tempo eles podem mudar de posição na sala sem necessariamente mudarem o local onde estão se reunindo.'}
{'frame1': 'frame_0001_time_1.00.png', 'frame2': 'frame_0002_time_2.00.png', 'descricao': ' Em um primeiro momento (instante 1), existe um grupo de pessoas sentadas numa sala. Passam-s

In [25]:
#@title Sumarizar a parte visual do video (LLaMa)
# Baseado em https://github.com/GoloMarcos/LLM4BrazilianFakeNews/blob/main/Evaluation.ipynb
!pip install ollama
import ollama

# Fazer a analise entre a descrição de dois frames
def analisaDescricoesFramesLlama(model, descricoes):
  # Criação do prompt para todas as descrições
  prompt = "Considere as seguintes descrições e resuma-as, contextualizando todas em conjunto. Explique como elas se relacionam/diferenciam e o que elas podem significar em conjunto:\n"

  # Adiconar cada descrição e sua respectiva timestamp
  for i, desc in enumerate(descricoes):
    prompt += f"\nDescrição {i + 1}: {desc['descricao']} instante {desc['frame1'].split('_time_')[1].split('.')[0]} até instante {desc['frame2'].split('_time_')[1].split('.')[0]}"

  prompt += "\nSeu resumo deve integrar todas as descrições e fornecer uma visão clara e coerente do contexto geral."

  response = ollama.chat(model=model, messages=[{
    'role': 'user',
    'content': prompt,
  },
  ])
  return response['message']['content']

# Criar a lista com as novas descrições
model = 'mistral'
def analiseFramesLlama(listDescricoes):
  listDescricoesVizinhos = []
  #listDescricoes = descricaoFrames() # Local já está com valor padrão correto

  if listDescricoes:
    # Chama a função de análise para todas as descrições de uma vez
    resposta = analisaDescricoesFramesLlama(model, listDescricoes)
    if resposta:
      print('Descrição final dos frames criada!')
    return resposta # Pode ser vazia
  else:
      return None

if novoResultadoContexto:
  sumarioParteVisual = analiseFramesLlama(novoResultadoContexto)
else:
  print('Nenhuma descrição encontrada!')

Descrição final dos frames criada!


In [26]:
print(sumarioParteVisual)

 As descrições analisadas mostram ocorrências repetidas de logotipos ou marcas com a palavra "eq" em sequência cronológica, indicando sua presença em diferentes momentos e locais. É possível que essas aparições sejam usadas como recurso visualmente chamativo para enfatizar a marca em questão ou como parte da identidade visual de algum produto ou empresa. Além disso, as repetições podem ser idênticas ou não, indicando duas ocorrências independentes ou repetições não-intencionais na mesma sequência de imagens. Por fim, é preciso entender um contexto específico para interpretar as descrições de forma correta e coerente.


In [27]:
#@title Criação do sumário final, sumarizar com a transcrição do audio e descrição dos frames (LLaMa)
# Baseado em https://github.com/GoloMarcos/LLM4BrazilianFakeNews/blob/main/Evaluation.ipynb
%pip install ollama
import ollama

# Fazer a analise entre a descrição de dois frames
def analiseFinal(model, descricaoVisual, descricaoAudio):
  # Criação do prompt para todas as descrições
  prompt = "Considere as seguintes descrições do visual e audio de um video e resuma-as, contextualizando todas em conjunto. Explique como elas se relacionam/diferenciam e o que elas podem significar em conjunto, destacando eventos importantes:\n"
  prompt += f'Parte visual do video: {descricaoVisual}\n'
  prompt += f'Audio do video: {descricaoAudio}\n'
  prompt += "\nSeu resumo deve integrar todas as descrições e fornecer uma visão clara e coerente do contexto geral."

  response = ollama.chat(model=model, messages=[{
    'role': 'user',
    'content': prompt,
  },
  ])
  return response['message']['content']


model = 'mistral'
if sumarioParteVisual and transcricaoAudio:
  sumarioFinal = analiseFinal(model, sumarioParteVisual, transcricaoAudio)
  print('Descrição final criada!')
else:
  sumarioFinal = None

Descrição final criada!


In [28]:
print(sumarioFinal)

 O vídeo apresenta imagens repetitivas de logotipos ou marcas com a palavra "eq" em diferentes momentos e locais, possivelmente para enfatizar sua presença ou como parte da identidade visual. Por outro lado, o áudio do vídeo é composto por discussões animadas sobre um conflito entre dois sites: "salfa, American meme" e "site dos menes". Os participantes expressam que o "site dos menes" é superior a qualquer coisa e mesmo que outros países estejam entrando em guerra por causa do "salta America memes", eles continuariam defendendo o "site dos menes". Essa discussão se tornou uma batalha entre seguidores dos dois sites, com algumas pessoas afirmando que tal debate não tem sentido pois ambos são bobos e a única coisa certa é o "site dos menes". A discussão se tornou tão intensa que alguém foi até mesmo assassinado apenas por falar a favor do "salta America memes". No final, um indivíduo anuncia que está saindo da discussão porque ela é bobo e não tem sentido.


Possíveis APIs para a 4ª parte: uso de BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies), Hugging Face Models

In [ ]:
# Q

5º Parte: Análise do sumário com dados do video